In [5]:
from llama_index import Document
from pathlib import Path
from llama_hub.file.pdf.base import PDFReader
from llama_index.response.notebook_utils import display_source_node
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI
import json

步骤1：加载文档
我们使用PDFReader加载PDF文件，并将文档的每一页合并为一个document对象。

In [6]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("gastrosis_papers.pdf"))
doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

设置openai秘钥

In [7]:
import openai
openai.api_key = ""
openai.api_base = "https://openkey.cloud/v1"

In [8]:
# 测试是否连通openai
def chat(message):
  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "user", "content": message}
      ],
      max_tokens=2500,
      temperature=0.2,
      top_p=0.75,
      frequency_penalty=0.0,
      presence_penalty=0.0
  )
  answer = response['choices'][0]['message']['content']
  return answer

chat("你是gpt3.5，还是gpt4？")

'我是OpenAI的GPT-3模型，不是GPT-4。'

步骤2：将文档解析为文本块（节点）
然后，我们将文档拆分为文本块，这些文本块在LlamaIndex中被称为“节点”，我们将chuck大小定义为1024。默认的节点ID是随机文本字符串，然后我们可以将节点ID格式化为遵循特定的格式。

In [9]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import IndexNode

In [10]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)
base_nodes = node_parser.get_nodes_from_documents(docs)
base_nodes[0]

TextNode(id_='3485a14e-8566-4723-bc92-c1d7cbc22352', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c39cf051-ac27-48a1-bda3-e449a76ce878', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='38d661b91cd9d4ceb7f0ebf3570365b96d5866ad798d01d13a27420fd57d9ff7'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='4a183bf8-b567-4f25-96f4-371aa1f7db02', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='ed343d520f210b5c993c2925a0124e2bf10ccd4e21e8ca028094e5f933bfce22')}, hash='9949d65a28e9c0819a6956f0328cc7b27cfc2d4309f96af58313c53d38351f1e', text='【\n摘\n要\n】\n幽\n门\n螺\n杆\n菌\n感\n染\n是\n一\n种\n感\n染\n性\n疾\n病\n，\n全\n球\n感\n染\n率\n高\n达\n5\n0\n%\n，\n其\n与\n消\n化\n不\n良\n、\n胃\n炎\n、\n消\n化\n性\n溃\n疡\n和\n胃\n癌\n的\n发\n生\n密\n切\n相\n关\n。\n根\n除\n幽\n门\n螺\n杆\n菌\n可\n有\n效\n控\n制\n相\n关\n疾\n病\n进\n展\n，\n降\n低\n相\n关\n疾\n病\n发\n生\n风\n险\n。\n然\n而\n，\n全\n球\n抗\n生\n素\n耐\n药\n率\n的\n升\n高\n使\n幽\n门\n螺\n杆\n菌\n感\

In [11]:
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [12]:
base_nodes[4]

TextNode(id_='node-4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c39cf051-ac27-48a1-bda3-e449a76ce878', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='38d661b91cd9d4ceb7f0ebf3570365b96d5866ad798d01d13a27420fd57d9ff7'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='69f46776-2b9f-4ff4-bc4a-f709a0332e41', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='a60e3666ac82da0328dcde9360f949e60b10748fc5f744a18d301067bc213230'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='1a2819c5-0450-4bc0-99ef-3b2b18e53084', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='f82c30fae39f4a9e1a1c488268021ab3cba0aae668c9db73139a5056ae5a1c91')}, hash='c04380bdae13646fd9cb647e398822531ea3971380eb5e7baf5333325b2ea9ac', text='指\n南\n制\n定\n组\n(\nG\nu\ni\nd\ne\nl\ni\nn\ne\nD\ne\nv\ne\nl\no\np\nm\ne\nn\nt\nG\nr\no\nu\np\n,\nG\nD\nG\n)\n专\n家\n成\n员\n构\n成\n和\n利\n益\

步骤3：选择embedding模型和LLM
我们需要定义两个模型：

embedding模型用于为每个文本块创建矢量嵌入。
LLM：用户查询和相关文本块喂给LLM，让其生成具有相关上下文的答案。
我们可以在ServiceContext中将这两个模型捆绑在一起，并在以后的索引和查询步骤中使用它们。

In [13]:
from llama_index.embeddings import resolve_embed_model
# 指定本地路径
local_embed_model_path = "./embed_model"

# 解析本地嵌入模型
embed_model = resolve_embed_model(f"local:{local_embed_model_path}")
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

/home/hujili/.conda/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


步骤4：创建索引、检索器和查询引擎
索引、检索器和查询引擎是基于用户数据或文档进行问答的三个基本组件：
1.索引是一种数据结构，使我们能够从外部文档中快速检索用户查询的相关信息。矢量存储索引获取文本块/节点，然后创建每个节点的文本的矢量嵌入，以便LLM查询。

In [14]:
base_index = VectorStoreIndex(base_nodes, service_context=service_context)

In [15]:
base_retriever = base_index.as_retriever(similarity_top_k=3)

In [16]:
retrievals = base_retriever.retrieve(
    "胃镜活检标本处理指南是？"
)

In [17]:
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-1237<br>**Similarity:** 0.6238844627865656<br>**Text:** 严
格
内
镜
检
查
前
准
备
(
咽
部
局
麻
、
链
霉
蛋
白
酶
、
西
甲
压
到
油
等
)
普
通
白
光
内
镜
检
查
染
色
、
放
大
内
镜
等
特
殊
技
术
检
查
内
镜
下
未
发
现
局
灶
病
变
内
镜
下
发
现
局
灶
病
变
多
处
取
活
检
取
活
检
萎
缩
或
肠
化
生
L
G
I
N
H
G
I
N
L
C
I
N
H
G
I
N
胃
癌
根
除
幽
门
螺
杆
菌
治
疗
内
镜
切
除
内
镜
切
除
或
外
科
手
术
内
镜
随
访
再
活
检
或
随
访
L
G
I
N
：
低
级
别
上
皮
内
瘤
变
每
3
年
精
查
每
年
精
查
每
6
~
1
2
个
月
检
查
H
G
I
N
：
高
级
别
上
皮
内
瘤
变
图
3
胃
癌
精
查
和
随
访
流
程
4
)
胃
镜
活
检
标
本
处
理
指
南
：
①
标
本
前
期
处
置
：
活
检
标
本
离
体
后
，
立
即
将
标
本
展
平
，
使
黏
膜
的
基
底
层
面
贴
附
在
滤
纸
上
。
②
标
本
固
定
：
置
于
充
足
(
>
1
0
倍
标
本
体
积
)
的
1
0
%
中
性
缓
冲
福
尔
马
林
溶
液
(
含
4
%
甲
醛
)
中
。
包
埋
前
固
定
时
间
须
>
6
h
,
<
4
8
h
。
③
石
蜡
包
埋
：
去
除
滤
纸
，
将
组
织
垂
直
定
向
包
埋
。<br>

**Node ID:** node-1251<br>**Similarity:** 0.5884364953210894<br>**Text:** 1
.
活
检
标
本
(
1
)
描
述
及
记
录
：
描
述
送
检
组
织
的
大
小
及
数
目
。
(
2
)
取
材
：
送
检
黏
膜
全
部
取
材
，
应
将
黏
膜
包
于
滤
纸
中
以
免
丢
失
，
取
材
时
应
滴
加
伊
红
，
利
于
包
埋
和
切
片
时
技
术
员
辨
认
。
大
小
相
差
悬
殊
的
要
分
开
放
入
不
同
脱
水
盒
，
防
止
小
块
活
检
组
织
漏
切
或
过
切
。
包
埋
时
需
注
意
一
定
要
将
展
平
的
黏
膜
立
埋
(
即
黏
膜
垂
直
于
包
埋
盒
底
面
包
埋
)
。
一
个
蜡
块
中
组
织
片
数
不
宜
超
过
3
片
、
平
行
方
向
立
埋
。
蜡
块
边
缘
不
含
组
织
的
白
边
尽
量
用
小
刀
去
除
，
建
议
每
张
玻
片
含
6
~
8
个
连
续
组
织
片
，
便
于
连
续
观
察
。
2
.
E
M
R
/
E
S
D
标
本
(
1
)
大
体
检
查
及
记
录
：
测
量
并
记
录
标
本
大
小
(
最
大
径
×
最
小
径
×
厚
度
)
，
食
管
胃
交
界
部
标
本
要
分
别
测
量
食
管
和
胃
的
长
度
和
宽
度
。<br>

**Node ID:** node-1011<br>**Similarity:** 0.5590414960434091<br>**Text:** 病
理
学
检
查
：
内
镜
医
师
可
根
据
需
求
决
定
活
检
取
材
标
本
数
(
一
般
为
2
~
5
块
)
和
部
位
。
如
取
5
块
活
检
组
织
时
，
则
胃
窦
2
块
(
分
别
取
自
距
幽
门
2
~
3
c
m
处
的
胃
大
弯
和
胃
小
弯
)
，
胃
体
2
块
(
分
别
取
自
距
贲
门
8
c
m
处
的
胃
大
弯
和
距
胃
角
口
侧
4
c
m
处
的
胃
小
弯
)
，
胃
角
1
块
。
标
本
应
足
够
大
，
并
且
达
到
黏
膜
肌
层
。
对
黏
膜
隆
起
、
凹
陷
、
变
色
等
可
见
病
灶
应
另
取
标
本
。
不
同
部
位
的
标
本
须
分
开
装
瓶
，
并
向
病
理
科
提
供
取
材
部
位
、
内
镜
下
所
见
和
简
要
病
史
[
1
]
。
病
理
活
检
结
果
示
固
有
腺
体
减
少
，
即
可
诊
断
C
A
G
[
1
]
。
病
理
科
医
师
需
观
察
胃
黏
膜
黏
液
层
、
表
面
上
皮
、
小
凹
上
皮
和
腺
管
上
皮
表
面
有
无
H
.
p
y
l
o
r
i
并
进
行
描
述
。
阿
利
新
蓝
-
过
碘
酸
希
夫
染
色
有
助
于
对
不
明
显
的
G
I
M
作
出
诊
断
。
在
显
微
镜
下
，
可
以
将
胃
黏
膜
萎
缩
、
H
.
p
y
l
o
r
i
感
染
和
G
I
M
分
为
轻
、
中
、
重
度
。
1
.<br>

In [18]:
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)
response = query_engine_base.query(
    "胃镜活检标本处理指南是？"
)
print(str(response))

胃镜活检标本处理指南包括以下步骤：
1. 描述和记录送检组织的大小和数量。
2. 取材：将送检黏膜全部取材，包裹在滤纸中以防丢失。取材时滴加伊红，有利于包埋和切片时技术员辨认。如果大小相差悬殊，要分开放入不同脱水盒，防止小块活检组织漏切或过切。包埋时要将展平的黏膜立埋，即黏膜垂直于包埋盒底面包埋。一个蜡块中组织片数不宜超过3片、平行方向立埋。蜡块边缘不含组织的白边尽量用小刀去除，建议每张玻片含6~8个连续组织片，便于连续观察。
3. 病理学检查：内镜医师根据需要决定活检取材标本数和部位。标本应足够大，并且达到黏膜肌层。对黏膜隆起、凹陷、变色等可见病灶应另取标本。不同部位的标本须分开装瓶，并向病理科提供取材部位、内镜下所见和简要病史。病理活检结果示固有腺体减少，即可诊断CAG。病理科医师需观察胃黏膜黏液层、表面上皮、小凹上皮和腺管上皮表面有无H.pylori并进行描述。阿利新蓝-过碘酸希夫染色有助于对不明显的GIM作出诊断。在显微镜下，可以将胃黏膜萎缩、H.pylori感染和GIM分为轻、中、重度。


步骤5：索引向量持久化到磁盘

In [19]:
# index 持久化到磁盘
base_index.storage_context.persist(persist_dir="./vector_index")

In [20]:
# 索引构建方法封装
def build_index(knowledge_base_dir: str) -> None:
    """Build the vector index from the pdf files in the directory."""
    print("Building vector index...")
    index = VectorStoreIndex(base_nodes, service_context=service_context)
    index.storage_context.persist(persist_dir=knowledge_base_dir)
    print("Index building completed.")

In [21]:
build_index("./vector_index")

Building vector index...
Done.
